In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
# Parametri
sequence_length = 10 #sequenza degli istanti temporali

#etichette per softmax
labels_coded = ['fermo', 'destra', 'sinistra', 'sopra', 'sotto', 'avanti', 'indietro']
labels = [0, 1, 2, 3, 4, 5, 6]

In [3]:
#data exploring
df = pd.read_csv('DATI.csv')

df

,timestamp,accX,accY,accZ,gyroX,gyroY,gyroZ,Label
0,1.732189e+09,-5.527879,-4.858718,-2.166334,-0.966667,-3.166667,-0.966667,"['sotto', 'fermo', 'fermo', 'fermo', 'sinistra..."
1,1.732189e+09,-5.200716,-5.413599,-2.259820,-0.366667,-2.866667,-1.400000,"['sotto', 'indietro', 'fermo', 'fermo', 'sinis..."
2,1.732189e+09,-3.524709,-5.594171,-1.247677,0.600000,-1.800000,-2.133333,"['fermo', 'indietro', 'fermo', 'fermo', 'sinis..."
3,1.732189e+09,-0.023709,-6.387939,1.155626,0.833333,-0.166667,-2.433333,"['fermo', 'indietro', 'fermo', 'fermo', 'fermo..."
4,1.732189e+09,2.153291,-6.756805,2.129320,0.600000,1.000000,-2.400000,"['fermo', 'indietro', 'fermo', 'fermo', 'fermo..."
...,...,...,...,...,...,...,...,...
40379,1.732189e+09,6.151457,-13.349115,13.103255,2.633333,2.133333,0.666667,"['sopra', 'indietro', 'destra', 'sinistra brac..."
40380,1.732189e+09,3.876457,-8.463914,11.822385,1.733333,0.833333,0.333333,"['fermo', 'indietro', 'destra', 'sinistra brac..."
40381,1.732189e+09,2.689457,-6.649928,11.716661,1.266667,0.133333,0.300000,"['fermo', 'indietro', 'destra', 'sinistra brac..."
40382,1.732189e+09,0.434457,-7.581928,12.356489,-0.266667,-0.066667,0.533333,"['fermo', 'indietro', 'destra', 'fermo', 'ferm..."


In [4]:
def trova_terza(str):
    for _ in range(3):
        idx = str.rfind(',')
        str = str[:idx]
    str = str + ']'
    print(str)

    return str

In [5]:
df['Label'] = df['Label'].apply(trova_terza)

['sotto', 'fermo', 'fermo']
['sotto', 'indietro', 'fermo']
['fermo', 'indietro', 'fermo']
['fermo', 'indietro', 'fermo']
['fermo', 'indietro', 'fermo']
['sopra', 'indietro', 'fermo']
['sopra', 'indietro', 'fermo']
['sopra', 'fermo', 'fermo']
['sopra', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'indietro', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'indietro', 'fermo']
['fermo', 'indietro', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'indietro', 'fermo']
['fermo', 'indietr

# aggregamento dati

In [6]:
from collections import Counter

def preprocess_data(df, colonna, batch_size=10):
    """
    Assegna la label più frequente ogni blocco di 10
    
    da specificare il df e la ['']

    """
    
    lista = []  
    
    for i in range(0, len(df), batch_size):
        batch = df.iloc[i:i + batch_size]  
        
        labels = batch[colonna]  #label del batch
        counter = Counter(labels)     #trova la label più frequente 
        most_common_label = counter.most_common(1)[0][0]
        
        lista.extend([most_common_label] * len(batch))
    
    return pd.Series(lista, index=df.index)

In [7]:
df['Label'] = preprocess_data(df, colonna='Label')

In [8]:
df

,timestamp,accX,accY,accZ,gyroX,gyroY,gyroZ,Label
0,1.732189e+09,-5.527879,-4.858718,-2.166334,-0.966667,-3.166667,-0.966667,"['fermo', 'indietro', 'fermo']"
1,1.732189e+09,-5.200716,-5.413599,-2.259820,-0.366667,-2.866667,-1.400000,"['fermo', 'indietro', 'fermo']"
2,1.732189e+09,-3.524709,-5.594171,-1.247677,0.600000,-1.800000,-2.133333,"['fermo', 'indietro', 'fermo']"
3,1.732189e+09,-0.023709,-6.387939,1.155626,0.833333,-0.166667,-2.433333,"['fermo', 'indietro', 'fermo']"
4,1.732189e+09,2.153291,-6.756805,2.129320,0.600000,1.000000,-2.400000,"['fermo', 'indietro', 'fermo']"
...,...,...,...,...,...,...,...,...
40379,1.732189e+09,6.151457,-13.349115,13.103255,2.633333,2.133333,0.666667,"['sotto', 'indietro', 'sinistra']"
40380,1.732189e+09,3.876457,-8.463914,11.822385,1.733333,0.833333,0.333333,"['fermo', 'indietro', 'destra']"
40381,1.732189e+09,2.689457,-6.649928,11.716661,1.266667,0.133333,0.300000,"['fermo', 'indietro', 'destra']"
40382,1.732189e+09,0.434457,-7.581928,12.356489,-0.266667,-0.066667,0.533333,"['fermo', 'indietro', 'destra']"


# train test split

In [9]:
# Caricamento e suddivisione dei dati
x = df.drop('Label', axis  = 1)
y = df['Label']

In [14]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [16]:
x_test

,timestamp,accX,accY,accZ,gyroX,gyroY,gyroZ
17777,1.732189e+09,3.854361,-3.535834,2.376260,4.500000,-1.766667,-0.300000
6049,1.732189e+09,-2.066824,-1.294614,0.024784,-2.733333,-0.233333,0.166667
25942,1.732189e+09,2.907118,-13.275489,-2.866574,3.800000,3.133333,5.633333
14505,1.732189e+09,3.039576,7.448282,5.818438,0.966667,-0.133333,1.166667
18169,1.732189e+09,3.245018,-17.377230,-11.362455,10.500000,-5.866667,-0.700000
...,...,...,...,...,...,...,...
16483,1.732189e+09,-0.171864,-8.887716,-7.489257,-1.933333,0.666667,0.166667
22503,1.732189e+09,0.046002,2.979538,-5.617897,-1.400000,-1.066667,0.966667
1721,1.732189e+09,0.797121,-8.512019,-1.743001,-3.733333,0.866667,-0.333333
14514,1.732189e+09,-1.515882,2.564829,0.995459,-0.866667,-1.566667,-2.933333


In [17]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
x_test  = scaler.fit_transform(x_test)
x_train  = scaler.fit_transform(x_train)

In [ ]:
# Creazione del modello LSTM
model = Sequential()
model.add(LSTM(64, input_shape=(sequence_length, 6), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32))
model.add(Dropout(0.2))
model.add(Dense(7, activation='softmax'))

# Compilazione del modello
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Allenamento
model.fit(x_train, y_train, epochs=5, batch_size=32, validation_split=0.2)


Epoch 1/5


c:\Users\NicoloCalcagno\miniconda3\envs\UFS12DL\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(None, 7), dtype=float32). Expected shape (None, 10, 7), but input has incompatible shape (None, 7)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 7), dtype=float32)
  • training=True
  • mask=None

In [ ]:
# Valutazione
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Loss: {loss}, Accuracy: {accuracy}")